In [5]:
!pip install -q -U google-generativeai

In [2]:
import pathlib
import textwrap

import google.generativeai as genai
import numpy as np
import pandas as pd

from IPython.display import display
from IPython.display import Markdown

In [3]:
from config import GOOGLE_API_KEY

genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [223]:
model = genai.GenerativeModel('gemini-pro')

In [251]:
%%time
response = model.generate_content(make_prompt("four", codes[8]))

CPU times: total: 0 ns
Wall time: 1.39 s


In [252]:
response = model.generate_content(make_prompt("four", codes[8]))
r = response.text

In [253]:
from io import StringIO
data_io = StringIO(r)
    
# Create first df to be cleaned and returned 
df = pd.read_csv(data_io, delimiter="|")
    
# Drop empty top row 
df = df.iloc[1:].reset_index(drop = True)


In [254]:
names = df.columns
for name in names: 
    if "unnamed" in name.lower(): 
        df = df.drop(columns = [name])
df

,Version 4 Code,Version 3.1 Code,Proportion
0,0122,0113,1.00


In [255]:
names = df.columns
if len(names) == 3 and len(df) != 0:
    df[names[2]] = df[names[2]].astype(str)
    props = np.array(df[names[2]])
    for i in range(len(props)): 
        if df[names[2]][i] == "**1**":
            df[names[2]][i] = "1.00"
        elif type(df[names[2]][i]) == str:
            if 'n/a' in df[names[2]][i].lower() or 'NaN' in df[names[2]][i].lower():
                df[names[2]][i] = "0"
    status = "clean"
    for i in range(len(props)):
        if "," in props[i] or "*" in props[i]:
            status = "broken"
status

'clean'

In [76]:
##Function takes the created prompt and returns a dataframe that has proportions showing how each 3 digit cade in ISIC 4 is created in ISIC 3.1

def get_proportions(prompt):
    from io import StringIO
    import csv
    import pathlib
    import textwrap
    import google.generativeai as genai
    import numpy as np
    import pandas as pd
    
    ## Need to change this line to get your own API key if possible
    from config import GOOGLE_API_KEY
    
    
    #setting the model and API key
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    
    # Input string prompt to the google model 
    response = model.generate_content(prompt)
    
    # Define the data string
    data_string = response.text
    
    # Create a StringIO object to mimic a file-like object
    data_io = StringIO(data_string)
    
    # Create first df to be cleaned and returned 
    df = pd.read_csv(data_io, delimiter="|")
    
    # Drop empty top row 
    df = df.iloc[1:].reset_index(drop = True)
    
    # Drop empty columns
    names = df.columns
    for name in names: 
        if "unnamed" in name.lower(): 
            df = df.drop(columns = [name])
    # Making proportion column a float between 0 and 1
    names = df.columns
    print(names)
    names_2 = ["version_4", "version_3.1", "Proportion of Jobs"]
    df.columns = names_2
    ## this is to ensure that the end of the data set is not a row that says "Total .... 1.00" which the prompt often returns
    ## This needs to be robust if the column type is string or float/int
    
    if type(df["version_4"].iloc[-1]) == str:
        if "total" in df["version_4"].iloc[-1].lower() or df["version_4"].iloc[-1].lower() == '': 
            df = df.iloc[:-1]
    elif df["Proportion of Jobs"].iloc[-1] == "**1.00**":
        if 
        df = df.iloc[:-1]
    elif type(df["version_3.1"].iloc[-1]) == str:
        if "total" in df["version_3.1"].iloc[-1].lower():
            df = df.iloc[:-1]
    else:
        if df["Proportion of Jobs"].iloc[-1] == 1 or "1.00" in df["Proportion of Jobs"].iloc[-1]:
            df = df.iloc[:-1]
    
     # Making proportion column a float between 0 and 1
    df["Proportion of Jobs"] = df["Proportion of Jobs"].str.strip()
    df["Proportion of Jobs"] = df["Proportion of Jobs"].str.replace("%", "")
    df["Proportion of Jobs"] = df["Proportion of Jobs"].astype(float) 
    if df["Proportion of Jobs"][0] > 1:
        df["Proportion of Jobs"] = df["Proportion of Jobs"]/100 
    
    # Replace non-numeric characters with an empty string
    pattern = r'\D'  # Matches any non-digit character
    df["version_4"] = df["version_4"].str.replace(pattern, '')    
    df["version_3.1"] = df["version_3.1"].str.replace(pattern, '')
    
    ## scale the data frame
    df = df.reset_index(drop = True)
    if df["Proportion of Jobs"].sum() > 1:
        total = df["Proportion of Jobs"].sum()
        for i in range(len(df["Proportion of Jobs"])):
            df["Proportion of Jobs"][i] = df["Proportion of Jobs"][i]/total
    return df

In [3]:
def step_one_cleaning(prompt):  
    from io import StringIO
    import csv
    import pathlib
    import textwrap
    import google.generativeai as genai
    import numpy as np
    import pandas as pd
    
    ## Need to change this line to get your own API key if possible
    from config import GOOGLE_API_KEY
    
    
    #setting the model and API key
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    
    # Input string prompt to the google model 
    response = model.generate_content(prompt)
    
    # Define the data string
    data_string = response.text
    
    # Create a StringIO object to mimic a file-like object
    data_io = StringIO(data_string)
    
    # Create first df to be cleaned and returned 
    df = pd.read_csv(data_io, delimiter="|")
    
    # Drop empty top row 
    df = df.iloc[1:].reset_index(drop = True)
    
    # Drop empty columns
    names = df.columns
    for name in names: 
        if "unnamed" in name.lower(): 
            df = df.drop(columns = [name])
    return df

In [22]:
def step_two_cleaning(prompt):
    import numpy as np
    import pandas as pd
    from collections import Counter
    status = "broken"
    while status == "broken":
        df = step_one_cleaning(prompt)
        names = df.columns
        if len(names) == 3 and len(df) != 0:
            count = Counter(df[names[2]][0])
            if count["."] == 1:
                df[names[2]] = df[names[2]].astype(str)
                props = np.array(df[names[2]])
                for i in range(len(props)): 
                    if df[names[2]][i] == "**1**":
                        df[names[2]][i] = "1.00"
                    elif type(df[names[2]][i]) == str:
                        if 'n/a' in df[names[2]][i].lower() or 'NaN' in df[names[2]][i].lower():
                            df[names[2]][i] = "0"
                status = "clean"
                for i in range(len(props)):
                    if "," in props[i] or "*" in props[i]:
                        status = "broken"
    # Making proportion column a float between 0 and 1
    names = df.columns
    names_2 = ["version_4", "version_3.1", "Proportion of Jobs"]
    df.columns = names_2
    ## this is to ensure that the end of the data set is not a row that says "Total .... 1.00" which the prompt often returns
    ## This needs to be robust if the column type is string or float/int
    
    if type(df["version_4"].iloc[-1]) == str:
        if "total" in df["version_4"].iloc[-1].lower() or df["version_4"].iloc[-1].lower() == '': 
            df = df.iloc[:-1]
    elif type(df["version_3.1"].iloc[-1]) == str:
        if "total" in df["version_3.1"].iloc[-1].lower():
            df = df.iloc[:-1]
    
    # Replace non-numeric characters with an empty string
    pattern = r'\D'  # Matches any non-digit character
    if type(df["version_4"]) == str:
        df["version_4"] = df["version_4"].str.replace(pattern, '') 
    if type(df["version_3.1"]) == str:
        df["version_3.1"] = df["version_3.1"].str.replace(pattern, '')
    
    # Making proportion column a float between 0 and 1
    if type(df["Proportion of Jobs"][0]) == str:
        df["Proportion of Jobs"] = df["Proportion of Jobs"].str.strip()
        df["Proportion of Jobs"] = df["Proportion of Jobs"].str.replace("%", "")
        df["Proportion of Jobs"] = df["Proportion of Jobs"].astype(float) 
    if df["Proportion of Jobs"][0] > 1:
        df["Proportion of Jobs"] = df["Proportion of Jobs"]/100 
    
    ## scale the data frame
    df = df.reset_index(drop = True)
    if df["Proportion of Jobs"].sum() > 1:
        total = df["Proportion of Jobs"].sum()
        for i in range(len(df["Proportion of Jobs"])):
            df["Proportion of Jobs"][i] = df["Proportion of Jobs"][i]/total
    return df

In [70]:
def test_proportions(prompt):
    from io import StringIO
    import csv
    import pathlib
    import textwrap
    import google.generativeai as genai
    import numpy as np
    import pandas as pd
    
    ## Need to change this line to get your own API key if possible
    from config import GOOGLE_API_KEY
    #setting the model and API key
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    
    # Input string prompt to the google model 
    response = model.generate_content(prompt)
    
    # Define the data string
    data_string = response.text
    
    # Create a StringIO object to mimic a file-like object
    data_io = StringIO(data_string)
    
    # Create first df to be cleaned and returned 
    df = pd.read_csv(data_io, delimiter="|")
    
    # Drop empty top row 
    df = df.iloc[1:].reset_index(drop = True)
    
    # Drop empty columns
    names = df.columns
    names = df.columns
    names_2 = []
    for name in names: 
        if "unnamed" not in name.lower(): 
            names_2.append(name)
    return names_2

In [5]:
## Creating a usable prompt 
## Start by importing the correspondence table and .txt files with with codes and descriptions
corr_table = pd.read_csv("ISIC4_ISIC31.csv", dtype={"ISIC4code": str, "partialISIC4": str, "ISIC31code": str, "partialISIC31": str})
corr_table = corr_table.fillna("")
ISIC_4 = pd.read_csv("isic4.txt", dtype={'num': str}, delimiter = "|")
ISIC_31 = pd.read_csv("isic31.txt", dtype={'num': str}, delimiter = "|")
col_names = ["code", "description"]
ISIC_4.columns = col_names
ISIC_31.columns = col_names

In [6]:
names = ISIC_31.columns 
data = {names[0]: ["6712"], names[1]: ["Security dealing activities"]}
df_small = pd.DataFrame(data)
ISIC_31 = pd.concat([ISIC_31, df_small])
ISIC_31 = ISIC_31.reset_index(drop = True)

In [213]:
for i in range(len(codes)):
    print(i)
    make_prompt("four", codes[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [212]:
ISIC_3_4digit[ISIC_3_4digit['code'] == "6712"]

,code,description
517,6712,Security dealing activities


In [7]:
ISIC_31.to_excel("ISIC_31.xlsx", index=True)

In [8]:
# Cleaning data and making dfs for each code length 
code_length_4 = ISIC_4['code'].str.len()
code_length_31 = ISIC_31['code'].str.len()

# Filter DataFrames based on code length
ISIC_4_2digit = ISIC_4[code_length_4 == 2]
ISIC_4_3digit = ISIC_4[code_length_4 == 3]
ISIC_4_4digit = ISIC_4[code_length_4 == 4]

ISIC_3_2digit = ISIC_31[code_length_31 == 2]
ISIC_3_3digit = ISIC_31[code_length_31 == 3]
ISIC_3_4digit = ISIC_31[code_length_31 == 4]

# Making 3 digit and 2 digit 4.0 codes columns in the correspondence table 
corr_table['ISIC_4_3d'] = corr_table['ISIC4code'].str[:3]
corr_table['ISIC_4_2d'] = corr_table['ISIC4code'].str[:2]

In [9]:
def make_prompt(digits, four_code):
    num_codes = corr_table["ISIC4code"].value_counts()
    prompt = "A " + digits + " digit code " + four_code + " which is (" + str(ISIC_4_4digit[ISIC_4_4digit["code"] == four_code]["description"].iloc[0]) + ") in ISIC version 4 is comprised of " + str(num_codes[four_code]) + " four digit codes in ISICs version 3.1, "
    codes_31 = corr_table[corr_table["ISIC4code"] == four_code]
    ## Code to handle when there are multiple instances of a code and possibly multiple "details" for that code in the correspondence table
    for code in codes_31["ISIC31code"].unique(): 
        ## start by just adding the code and its standard description from the ISIC code data frame
        prompt = prompt + code + " which is (" + ISIC_3_4digit[ISIC_3_4digit['code'] == code]['description'].iloc[0] + "), "
        ## now test if the code is unique, that is, it only appears once within the 3 digit code we are considering from version 4
        if codes_31['ISIC31code'].eq(code).sum() == 1:
            ## If it is unique, we test if the detail column of the correspondence table is empty, if it is not we add that extra detail to the prompt
            if codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[0] != "":
                prompt = prompt + " and includes (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[0] + ") "
        elif codes_31['ISIC31code'].eq(code).sum() > 1:
            ## we now test if the code is not unique, if this is the case our code is more complicated, start by making an index list that will track appearances of this code
            ind = []
            ## Now we will loop through the instances of the code and add the index of each instance if the detail section is not empty, we are collecting all the different details to include 
            for i in range(codes_31['ISIC31code'].eq(code).sum()):
                if codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[i] != "":
                    ind.append(i)
            ## If the code is repeated but never with a detail the prompt remains unchanged
            if len(ind) == 0:
                prompt = prompt
            ## If the code is repeated but only with 1 detail, just include that single detail
            elif len(ind) == 1:
                prompt = prompt + "and includes (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[ind[0]] + ", "
            ## If the code is repeated with multiple details, start with the first detail and drop the first item in the list so it is not repeated, then loop through the list adding all the details
            else: 
                prompt = prompt + " and includes (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[ind[0]]
                ind = ind[1:]
                for k in ind:
                    prompt = prompt + ", and (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[k] + ")"
    if num_codes[four_code] > 10:
        prompt = prompt + " What is your best estimate of the proportion of jobs now coded in " + four_code + " that were in each of the previous codes in version 3.1? The proportions can be less than .05 and many probably are less .05. Can you make sure these proportions all sum to 1? Can you give me your best guesses in a table with 3 columns, first the three digit code, " + four_code + " then the four didgit codes, then the proportions?"
    else: 
        prompt = prompt + " What is your best estimate of the proportion of jobs now coded in " + four_code + " that were in each of the previous codes in version 3.1? If there is only 1 version 3.1 code, it is automatically 1. Can you give me your best guesses in a table with 3 columns, first the version 4 code, " + four_code + " then the version 3.1 codes, then the proportions?"
    return prompt

In [24]:
codes = np.array(ISIC_4_4digit["code"])
df_big = pd.DataFrame({'version_4': [], 'version_3.1': [], "Proportion of Jobs": []})
i = 0
for four_code in codes:
    prompt = make_prompt("four", four_code)
    df = step_two_cleaning(prompt)
    i += 1
    print(i)
    df_big = pd.concat([df_big, df])
    df_big = df_big[df_big.notna().all(axis=1)]
    df_big = df_big.reset_index(drop = True)
df_big.to_excel("correspondence_table.xlsx", index=True)
print(df_big)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


DeadlineExceeded: 504 Deadline Exceeded

In [13]:
df_big.to_excel("correspondence_table.xlsx", index=True)

In [17]:
from collections import Counter
#my_string = "Hello, world!"
#char_counts = Counter(my_string)
char_counts["l"]

3